#  WeatherPy

In [53]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [54]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
cities = len(cities)
cities

599

# Perform API Calls

Perform a weather check on each city using a series of successive API calls.
Include a print log of each city as it'sbeing processed (with the city number and city name).

In [55]:
# Establish base URL
url = "http://api.openweathermap.org/data/2.5/weather?"
city = "Boston"


# Build query URL
query_url = url + "q=" + city + "&units=imperial" + "&appid=" + weather_api_key

In [58]:
weather_response = requests.get(query_url)
weather_json = weather_response.json()
weather_json

{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 27.52,
  'feels_like': 10.89,
  'temp_min': 26.01,
  'temp_max': 28.99,
  'pressure': 1014,
  'humidity': 31},
 'visibility': 10000,
 'wind': {'speed': 18.41, 'deg': 290, 'gust': 27.63},
 'clouds': {'all': 75},
 'dt': 1611527805,
 'sys': {'type': 1,
  'id': 3486,
  'country': 'US',
  'sunrise': 1611489894,
  'sunset': 1611524856},
 'timezone': -18000,
 'id': 4930956,
 'name': 'Boston',
 'cod': 200}

In [59]:
# Create empty lists to hold the temperature, humidity, cloudiness, and wind speeds of each city 
max_temperature = []
humidity = []
cloudiness = []
wind_speed = []
latitude = []
longitude = []

for city in range(cities):
    weather_response = requests.get(query_url).json()
    
    try: 
        max_temperature.append(weather_response)['main']['temp_max']
        humidity.append(weather_reponse)['main']['humidity']
        cloudiness.append(weather_response)['clouds']['all']
        wind_speed.append(weather_response)['wind']['speed']
        latitude.append(weather_response)['coord']['lat']
        longitude.append(weather_response)['coord']['lon']
        print(f"Processing Record {city}")
        
    except:
        print("City not found")

City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not f

City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found


# Convert Raw Data to DataFrame

Export the city data into a .csv.
Display the DataFrame

In [ ]:
weather_df = pd.DataFrame({'City': cities,
                          'Temperature': temperature,
                          'Humidity': humidity,
                          'Cloudiness': cloudiness,
                          'Wind': wind})
weather_df

#  Inspect the data and remove the cities where the humidity > 100%.
Skip this step if there are no cities that have humidity > 100%.

In [6]:
#  Get the indices of cities that have humidity over 100%.

In [61]:
Int64Index([weather_df], dtype='int64') 

NameError: name 'Int64Index' is not defined

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".

# Plotting the Data

Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
Save the plotted figures as .pngs.

#  Latitude vs. Temperature Plot

#  Latitude vs. Humidity Plot

# Latitude vs. Cloudiness Plot

# Latitude vs. Wind Speed Plot

#  Linear Regression

#  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

# Southern Hemisphere - Max Temp vs. Latitude Linear Regression

# Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

# Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

# Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

#  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

# Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

#  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression